<a href="https://colab.research.google.com/github/cavs1010/personal-projects/blob/main/GC_Real_Estate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Data Collection

The data will be collected from https://www.domain.com.au/rent/gold-coast-qld/ <br> I tried using www.realestate.com.au but there is not permission.

In [ ]:
! pip3 install requests
! pip3 install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math

In [ ]:
URL = 'https://www.domain.com.au/rent/gold-coast-qld/'
page = requests.get(URL)

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
#Number of web pages to read
numberOfWebsToRead = soup.find("h1", attrs={"class": "css-ekkwk0"}).text
numberOfWebsToRead = int(re.findall(r'\d+', numberOfWebsToRead)[0])
numberOfWebsToRead = numberOfWebsToRead /21
numberOfWebsToRead = math.floor(numberOfWebsToRead)

In [ ]:
 #Arrays for the ammenities in the house
 mainAddressArray = []
 secondAddressArray = []
 postalCodeArray = []
 rentArray = []
 bedsArray = []
 bathsArray = []
 parkingArray = []
 buildingTypeArray = []

for i in range (1,7):
  URL = 'https://www.domain.com.au/rent/gold-coast-qld/?page='+str(i)
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  #Each element will cotain the info required for my work
  elements = soup.find_all("div", class_="css-1n74r2t")
  for element in elements:
    mainAddress = element.find("span", attrs={"data-testid": "address-line1"})
    secondAddress = element.find("span", attrs={"itemprop": "addressLocality"})
    postalCode = element.find("span", attrs={"itemprop": "postalCode"})
    rent = element.find("p", attrs={"data-testid": "listing-card-price"})
    ammenities = element.find_all("span", attrs={"class": "css-1rzse3v"})
    buildingType = element.find("span", attrs={"class": "css-693528"})
    mainAddressArray.append(mainAddress.text)
    secondAddressArray.append(secondAddress.text)
    postalCodeArray.append(postalCode.text)
    rentArray.append(rent.text)
    bedsArray.append(ammenities[0].text)
    bathsArray.append(ammenities[1].text)
    parkingArray.append(ammenities[2].text)
    buildingTypeArray.append(buildingType.text)

In [ ]:
dictionary = {'Main Address': mainAddressArray, 
              'Second Address': secondAddressArray, 
              'Postal Code': postalCodeArray, 
              'Rent':rentArray, 
              'Bedrooms': bedsArray, 
              'Bathrooms': bathsArray,
              'Parking spaces': parkingArray,
              'Building Type': buildingTypeArray}
df = pd.DataFrame(dictionary)
df

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"6 Cambridge Circuit,",PIMPAMA,4209,$490 Weekly,4 .css-9fxapx{position:absolute;width:1px;heig...,2 Baths,2 Parking,House
1,"1/24 McMillan Street,",LABRADOR,4215,$480,3 Beds,2 Baths,2 Parking,Duplex
2,"161 Musgrave Avenue,",LABRADOR,4215,$550 per week with Pool!!,4 Beds,2 Baths,2 Parking,House
3,"3035/2623-2633 Gold Coast Highway,",BROADBEACH,4218,$520 Per Week,1 Bed,1 Bath,1 Parking,Apartment / Unit / Flat
4,"8 Kiamba Street,",UPPER COOMERA,4209,$500,3 Beds,2 Baths,2 Parking,House
...,...,...,...,...,...,...,...,...
78,"28/26 Monaco Street,",SURFERS PARADISE,4217,$420,2 Beds,2 Baths,1 Parking,Apartment / Unit / Flat
79,"3/57 Garrick Street,",COOLANGATTA,4225,$485,2 Beds,1 Bath,1 Parking,Apartment / Unit / Flat
80,"17D Village Boulevard,",SOUTHPORT,4215,2 bedrooms from $435 per week,2 Beds,2 Baths,1 Parking,Apartment / Unit / Flat
81,"6/14 Brighton Street,",BIGGERA WATERS,4216,$390,2 .css-9fxapx{position:absolute;width:1px;heig...,1 Bath,1 Parking,Apartment / Unit / Flat


# 2. Data Cleaning

In [ ]:
dfCopy = df.copy()

In [ ]:
#Function that transform the columns[Bedrooms, Bathrooms and Parking Spaces] to integers
#columnName : Name of the column to transform into an integer.
def toInteger (columnName):
  for ind in dfCopy[columnName].index:
    value = dfCopy[columnName][ind][0]
    if (value == '−'):
      value = 0
    else:
      value = int(value)
    dfCopy[columnName][ind] = value

In [ ]:
toInteger('Bathrooms')
toInteger('Bedrooms')
toInteger('Parking spaces')
dfCopy

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"6 Cambridge Circuit,",PIMPAMA,4209,$490 Weekly,4,2,2,House
1,"1/24 McMillan Street,",LABRADOR,4215,$480,3,2,2,Duplex
2,"161 Musgrave Avenue,",LABRADOR,4215,$550 per week with Pool!!,4,2,2,House
3,"3035/2623-2633 Gold Coast Highway,",BROADBEACH,4218,$520 Per Week,1,1,1,Apartment / Unit / Flat
4,"8 Kiamba Street,",UPPER COOMERA,4209,$500,3,2,2,House
...,...,...,...,...,...,...,...,...
78,"28/26 Monaco Street,",SURFERS PARADISE,4217,$420,2,2,1,Apartment / Unit / Flat
79,"3/57 Garrick Street,",COOLANGATTA,4225,$485,2,1,1,Apartment / Unit / Flat
80,"17D Village Boulevard,",SOUTHPORT,4215,2 bedrooms from $435 per week,2,2,1,Apartment / Unit / Flat
81,"6/14 Brighton Street,",BIGGERA WATERS,4216,$390,2,1,1,Apartment / Unit / Flat


In [ ]:
rentIntArray = []
for ind in dfCopy['Rent'].index:
  #Extract the numbers from the string
  lista = re.findall(r'\d+', dfCopy['Rent'][ind])
  finalRent = ''.join(lista)
  #Eliminate the row if the number is ''
  if (finalRent == ''):
    dfCopy.drop([ind], inplace=True)
    print ('This is the problem index: '+str(ind)+', so it was eliminated' )
  #Otherwise, it add the number to a list
  else:
    dfCopy['Rent'][ind] = float(finalRent)
    rentIntArray.append(float(finalRent))

This is the problem index: 64, so it was eliminated


In [ ]:
dfCopy.head(40)

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"6 Cambridge Circuit,",PIMPAMA,4209,490,4,2,2,House
1,"1/24 McMillan Street,",LABRADOR,4215,480,3,2,2,Duplex
2,"161 Musgrave Avenue,",LABRADOR,4215,550,4,2,2,House
3,"3035/2623-2633 Gold Coast Highway,",BROADBEACH,4218,520,1,1,1,Apartment / Unit / Flat
4,"8 Kiamba Street,",UPPER COOMERA,4209,500,3,2,2,House
5,"17/120 The Esplanade,",SURFERS PARADISE,4217,450,2,1,1,Apartment / Unit / Flat
6,"5 Lakeridge Drive,",VARSITY LAKES,4227,6707,3,2,2,House
7,"2/44 Coolangatta Road,",COOLANGATTA,4225,440,2,1,1,Apartment / Unit / Flat
8,"25 Summerland Key,",BROADBEACH WATERS,4218,1450,4,2,2,House
9,"26 Springsure Drive,",MUDGEERABA,4213,650,3,2,2,House


## Get the latitude and Longitude

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")
city ="London"
country ="Uk"
loc = geolocator.geocode(city+','+ country)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 51.5073219 
longtitude is:- -0.1276474


In [ ]:
housesNotFound = []
for index, row in dfCopy.iterrows():
  if '/' in row['Main Address']:
    location =  geolocator.geocode(row['Main Address'].split('/',1)[1] +row['Second Address']) 
  else:
    location = geolocator.geocode(row['Main Address']+row['Second Address']) 
  if location == None:
    housesNotFound.append( row['Main Address']+row['Second Address'])
    dfCopy.drop([index], inplace=True)
  else:
    print (str(index))
    print ([location.latitude, location.longitude])
    dfCopy['Latitude'] = location.latitude
    dfCopy['Longitude'] = location.longitude

0
[-27.8355621, 153.3150188]
1
[-27.9484566, 153.3887535]
2
[-27.9563324, 153.4010099]
4
[-27.8637718, 153.2873029]
5
[-28.0037055, 153.4312287]
6
[-28.090281, 153.4198842]
7
[-28.1670021, 153.5212003]
8
[-28.0277253, 153.4206513]
9
[-28.0897004, 153.3743755]
11
[-28.0377118, 153.424929]
12
[-28.0260439, 153.3981841]
13
[-27.9514383, 153.3771857]
14
[-28.047423, 153.4222889]
15
[-28.0004615, 153.4221523]
16
[-28.0037055, 153.4312287]
17
[-28.0796875, 153.397988]
19
[-27.9759813, 153.4162493]
20
[-28.0030322, 153.4288667]
21
[-28.088257, 153.4236477]
22
[-28.048501, 153.3610459]
23
[-28.0007209, 153.315282]
24
[-27.8730903, 153.2917017]
25
[-28.0312426, 153.4164519]
26
[-27.9907356, 153.3952517]
27
[-27.9724441, 153.4026842]
28
[-28.0048577, 153.4233589]
29
[-27.9705434, 153.36483615674967]
30
[-28.0469567, 153.4175161]
31
[-27.8445534, 153.3376485]
32
[-28.0760916, 153.3555424]
33
[-27.9940446, 153.4260426]
34
[-27.9762084, 153.4175585]
35
[-27.8370707, 153.2963964]
36
[-28.1030376, 15

In [ ]:
housesNotFound

['53 Amalfi Drive,\xa0ISLE OF CAPRI',
 '14 Mersey Street,\xa0UPPER COOMERA',
 '1873 Sawgrass Place,\xa0SANCTUARY COVE',
 '44 Edinburgh Road,\xa0BENOWA WATERS',
 '702/6 Tonga Place,\xa0SOUTHPORT',
 '102 Cabana Boulevard,\xa0BENOWA WATERS',
 "95 'Lake Hills' 117 Palm Meadows Drive,\xa0CARRARA"]

In [ ]:
dfCopy

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type,Latitude,Longitude
0,"6 Cambridge Circuit,",PIMPAMA,4209,490,4,2,2,House,-27.933315,153.405034
1,"1/24 McMillan Street,",LABRADOR,4215,480,3,2,2,Duplex,-27.933315,153.405034
2,"161 Musgrave Avenue,",LABRADOR,4215,550,4,2,2,House,-27.933315,153.405034
4,"8 Kiamba Street,",UPPER COOMERA,4209,500,3,2,2,House,-27.933315,153.405034
5,"17/120 The Esplanade,",SURFERS PARADISE,4217,450,2,1,1,Apartment / Unit / Flat,-27.933315,153.405034
...,...,...,...,...,...,...,...,...,...,...
77,"6 Ulrike Way,",BENOWA,4217,900,4,3,2,House,-27.933315,153.405034
78,"28/26 Monaco Street,",SURFERS PARADISE,4217,420,2,2,1,Apartment / Unit / Flat,-27.933315,153.405034
79,"3/57 Garrick Street,",COOLANGATTA,4225,485,2,1,1,Apartment / Unit / Flat,-27.933315,153.405034
80,"17D Village Boulevard,",SOUTHPORT,4215,2435,2,2,1,Apartment / Unit / Flat,-27.933315,153.405034


## Map creation

In [ ]:
! pip install folium

In [ ]:
import folium

In [ ]:
gcMap = folium.Map(location=[-28.016, 153.4], zoom_start = 12, tiles = 'https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png')

ValueError: ignored

In [ ]:
gcMap